In [1]:
import pandas as pd
import pytz

In [17]:
# Read dataframes with datetime index
## Emissions
emissions = pd.read_csv(
    '../data/interim/emissions_germany_utc_202212312200_202412312200.csv',
    sep=',',
    index_col=0
)

## Generation
regions = {
    'f_hertz': pd.read_csv(
        '../data/interim/generation_f_hertz_utc_202212312300_202412312245.csv',
        sep=',',
        index_col=0
    ),
    'amprion': pd.read_csv(
        '../data/interim/generation_amprion_utc_202212312300_202412312245.csv',
        sep=',',
        index_col=0
    ),
    'tennet': pd.read_csv(
        '../data/interim/generation_tennet_utc_202212312300_202412312245.csv',
        sep=',',
        index_col=0
    ),
    'transnet_bw': pd.read_csv(
        '../data/interim/generation_transnet_bw_utc_202212312300_202412312245.csv',
        sep=',',
        index_col=0
    )
}

# Convert index to datetime
for df in (emissions, *regions.values()):
    df.index = pd.to_datetime(df.index, format='ISO8601')
    df.sort_index(inplace=True)
    # Check ich any timezone is set - if not, all the same
    if df.index.tz is not None:
        print(f'Timezone set to {df.index.tz}')

print(f"Emissions duplicates: {emissions.index.duplicated().sum()}")
for reg in regions:
    print(f"Region Duplicates: {regions[reg].index.duplicated().sum()}")

Timezone set to UTC
Timezone set to UTC
Timezone set to UTC
Timezone set to UTC
Timezone set to UTC
Emissions duplicates: 0
Region Duplicates: 0
Region Duplicates: 0
Region Duplicates: 0
Region Duplicates: 0


In [19]:
# Regional allocation of emissions based on share of regional generation from total generation
## Aggregate total generation per production type and the hour
total_gen_15min = pd.concat(regions.values()).groupby(level=0).sum()
total_gen_hourly = total_gen_15min.resample('1h').sum()

## Allocate emissions to regions based on share of regional generation
regional_emissions_final = {}

for name, df_reg in regions.items():
    fuels = ['lignite', 'hard_coal', 'fossile_gas', 'other_conventionals']
    regional_emissions_15min = pd.DataFrame(index=df_reg.index)

    for fuel in fuels:
        if fuel in df_reg.columns:
            ## (1) Regional hourly generation per production type
            regional_gen_hourly = df_reg[fuel].resample('h').sum()

            ## Share of regional generation per production type on total generation per production type
            regional_share_h = (regional_gen_hourly / total_gen_hourly[fuel]).fillna(0) # In case of no generation in a region, set share to 0

            ## Regional emissions per hour
            regional_emissions_hourly = emissions[fuel] * regional_share_h

            ## (2) Temporal downscaling to 15 min
            denom = df_reg[fuel].resample('h').transform('sum')
            weights = (df_reg[fuel] / denom).fillna(0.25) # share of each quarter-hour on the generation per hour; uniform distribution in case of no generation

            ## Regional emissions per quarter-hour
            regional_emissions_15min[fuel] = regional_emissions_hourly.resample('15min').ffill() * weights # weighted upsampling of regional emissions to 15 min resolution

    # Total emissions per control area
    regional_emissions_15min['total_emission']  = regional_emissions_15min.sum(axis=1)
    regional_emissions_final[name] = regional_emissions_15min


In [22]:
total_gen_hourly.head()

,lignite,hard_coal,fossile_gas,other_conventionals,total_generation
datetime,,,,,
2022-12-31 23:00:00+00:00,3859.75,2067.75,1593.75,1880.75,9402.00
2023-01-01 00:00:00+00:00,3866.00,2051.50,1436.50,1847.50,9201.50
2023-01-01 01:00:00+00:00,3860.00,2034.00,1434.75,1783.00,9111.75
2023-01-01 02:00:00+00:00,3864.25,2037.25,1432.75,1791.50,9125.75
2023-01-01 03:00:00+00:00,3840.50,2040.00,1430.50,1819.00,9130.00


In [23]:
regional_emissions_final['amprion'].head()

,lignite,hard_coal,fossile_gas,other_conventionals,total_emission
datetime,,,,,
2022-12-31 23:00:00+00:00,390.550541,172.897180,196.885227,264.010145,1024.343094
2022-12-31 23:15:00+00:00,390.002785,174.159203,196.885227,256.563705,1017.610920
2022-12-31 23:30:00+00:00,389.181150,174.159203,201.030180,256.563705,1020.934238
2022-12-31 23:45:00+00:00,390.002785,173.528192,198.543208,253.517434,1015.591619
2023-01-01 00:00:00+00:00,393.383775,172.759337,147.228625,255.879575,969.251312


In [25]:
regional_emissions_final['f_hertz'].head()

,lignite,hard_coal,fossile_gas,other_conventionals,total_emission
datetime,,,,,
2022-12-31 23:00:00+00:00,662.237874,94.862078,186.108352,241.670825,1184.879129
2022-12-31 23:15:00+00:00,663.607266,94.020729,186.108352,241.332350,1185.068698
2022-12-31 23:30:00+00:00,667.989320,93.179381,186.522847,243.024723,1190.716271
2022-12-31 23:45:00+00:00,667.167685,92.548369,186.108352,239.978452,1185.802858
2023-01-01 00:00:00+00:00,662.555462,92.264415,207.040254,244.201850,1206.061981


In [26]:
regional_emissions_final['tennet'].head()

,lignite,hard_coal,fossile_gas,other_conventionals,total_emission
datetime,,,,,
2022-12-31 23:00:00+00:00,1.917149,76.773079,198.543208,94.772873,372.006308
2022-12-31 23:15:00+00:00,1.917149,76.773079,198.543208,94.772873,372.006308
2022-12-31 23:30:00+00:00,1.917149,76.773079,198.543208,94.772873,372.006308
2022-12-31 23:45:00+00:00,1.917149,76.773079,198.543208,94.772873,372.006308
2023-01-01 00:00:00+00:00,1.910955,76.081363,219.002580,96.169505,393.164402


In [27]:
regional_emissions_final['transnet_bw'].head()

,hard_coal,fossile_gas,other_conventionals,total_emission
datetime,,,,
2022-12-31 23:00:00+00:00,90.444997,77.510605,42.647793,210.603395
2022-12-31 23:15:00+00:00,90.865671,77.510605,42.647793,211.024069
2022-12-31 23:30:00+00:00,91.076009,77.510605,42.647793,211.234407
2022-12-31 23:45:00+00:00,90.865671,77.510605,42.647793,211.024069
2023-01-01 00:00:00+00:00,90.162720,86.036728,43.276277,219.475725
